In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [92]:
sources = {
    "litoral": {
        "violencia_narco": "https://www.ellitoral.com/tag/violencia-narco",
        "narcotrafico": "https://www.ellitoral.com/tag/narcotrafico",
    },
    "aire": {
        "droga": "https://www.airedesantafe.com.ar/droga-a848/",
        "narcotrafico": "https://www.airedesantafe.com.ar/narcotrafico-a534",
        "cocaina": "https://www.airedesantafe.com.ar/cocaina-a1378",
        "drogas": "https://www.airedesantafe.com.ar/drogas-a4787",
    },
    "rosario12": {
        "general": "https://www.pagina12.com.ar/suplementos/rosario12/{date}",
    },
}

In [102]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException, 
    ElementNotInteractableException, 
    ElementClickInterceptedException
)
from bs4 import BeautifulSoup
import pandas as pd

def scrape_with_multiple_clicks_aire(sources):
    # Path to your Brave executable
    brave_path = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

    # Set up Chrome options
    options = Options()
    options.binary_location = brave_path

    # Create a WebDriver instance
    driver = webdriver.Chrome(options=options)  

    # List to store all scraped data
    data = []

    # Extract URLs and tags
    urls = list(sources["aire"].values())
    tags = list(sources["aire"].keys())

    try:
        # Iterate through each URL and tag
        for url, tag in zip(urls, tags):
            try:
                # Load the webpage
                driver.get(url)
                original_url = driver.current_url  # Store the original URL
                print(f"Processing URL: {url}")

                # Wait for the page to load initially
                time.sleep(5)

                # Handle cancel buttons if present
                try:
                    cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                    cancel_button.click()
                    time.sleep(2)
                except NoSuchElementException:
                    print("Cancel button not found.")
                
                # Start scraping loop
                while True:
                    # Extract page content
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    divs = soup.find_all('div', class_='article-title')
                    
                    for div in divs:
                        a_tag = div.find('a', class_='a-article-link')
                        if a_tag:
                            href = a_tag['href']
                            title = a_tag.get_text(strip=True)
                            data.append({'Link': href, 'Title': title, 'Tag': tag, 'Media': 'aire'})

                    print(f"Scraped {len(data)} links so far.")
                            
                    # Click the next button
                    try:
                        next_button = driver.find_element(By.CSS_SELECTOR, ".next.show a")
                        driver.execute_script("arguments[0].scrollIntoView();", next_button)
                        next_button.click()
                        time.sleep(5)  # Wait for new content to load
                    except NoSuchElementException:
                        print("Next button not found. Exiting loop.")
                        break
                    except (ElementNotInteractableException, ElementClickInterceptedException) as e:
                        print(f"Button click error: {e}")
                        break

                    # Stop if redirected back to the original URL
                    if driver.current_url == original_url:
                        print("Redirected back to the original URL. Breaking loop.")
                        break

                    # Handle cancel buttons again if they reappear
                    try:
                        cancel_button = driver.find_element(By.ID, "onesignal-slidedown-cancel-button")
                        cancel_button.click()
                        time.sleep(2)
                    except NoSuchElementException:
                        pass

            except Exception as e:
                print(f"Error processing {url}: {e}")

    finally:
        # Close the WebDriver
        driver.quit()

    # Create a DataFrame from the data list
    return pd.DataFrame(data)


In [103]:
import warnings

warnings.filterwarnings("ignore")

df_aire = scrape_with_multiple_clicks_aire(sources)

Processing URL: https://www.airedesantafe.com.ar/droga-a848/
Scraped 10 links so far.
Scraped 20 links so far.
Scraped 30 links so far.
Scraped 40 links so far.
Scraped 50 links so far.
Scraped 60 links so far.
Scraped 70 links so far.
Scraped 80 links so far.
Scraped 90 links so far.
Scraped 100 links so far.
Scraped 110 links so far.
Scraped 120 links so far.
Scraped 130 links so far.
Scraped 133 links so far.
Next button not found. Exiting loop.
Processing URL: https://www.airedesantafe.com.ar/narcotrafico-a534
Cancel button not found.
Scraped 143 links so far.
Scraped 153 links so far.
Scraped 163 links so far.
Scraped 173 links so far.
Scraped 183 links so far.
Scraped 193 links so far.
Scraped 203 links so far.
Scraped 213 links so far.
Scraped 223 links so far.
Scraped 233 links so far.
Scraped 243 links so far.
Scraped 253 links so far.
Scraped 263 links so far.
Scraped 273 links so far.
Scraped 283 links so far.
Scraped 293 links so far.
Scraped 303 links so far.
Scraped 313 l

In [105]:
df_aire.to_excel("data/df_aire_historic.xlsx", index=False)

In [109]:
df_aire["Link"][0]

'https://www.airedesantafe.com.ar/policiales/a-un-ano-la-ley-microtrafico-santa-fe-se-derrumbaron-45-bunkeres-droga-y-hubo-mas-100-condenas-toda-la-provincia-n604894'

In [129]:
import requests
from bs4 import BeautifulSoup

def content_and_date_extract(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content.decode('utf-8', errors='ignore'), 'html.parser')
        
        # Extract paragraphs
        paragraphs = soup.findAll('p')
        content_list = [p.get_text() for p in paragraphs if p.get_text()]
        
        # Extract the <div> with the class 'article-date'
        date_div = soup.find('div', class_='article-date')

        # Extract datetime attribute
        if date_div:
            # Find the <time> tag within that div
            datetime_element = date_div.find('time', attrs={"datetime": True})
            if datetime_element:
                extracted_date = datetime_element['datetime']  # Extract the value of the datetime attribute
            else:
                extracted_date = "Date not found in time tag"
        else:
            extracted_date = "No article-date div found"

        # Return a dictionary with both content and date
        return {"content": content_list if content_list else ["No content found"], "date": extracted_date}
    
    except requests.exceptions.RequestException as e:
        return {"content": [f"Request error: {e}"], "date": None}
    except Exception as e:
        return {"content": [f"Error: {e}"], "date": None}

# Apply the function to the 'Link' column
df_extracted = df_aire['Link'].apply(content_and_date_extract)

# Create separate columns for 'content' and 'date'
df_aire['content'] = df_extracted.apply(lambda x: x['content'])
df_aire['date'] = df_extracted.apply(lambda x: x['date'])


In [132]:
df_aire.to_excel("data/df_aire_historic_content.xlsx", index=False)

In [119]:
import requests
from bs4 import BeautifulSoup

def test_date_extraction(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content.decode('utf-8', errors='ignore'), 'html.parser')
        
        # Extract the <div> with the class 'article-date'
        date_div = soup.find('div', class_='article-date')
        
        if date_div:
            # Find the <time> tag within that div
            datetime_element = date_div.find('time', attrs={"datetime": True})
            if datetime_element:
                extracted_date = datetime_element['datetime']  # Extract the value of the datetime attribute
                return extracted_date
            else:
                return "Date not found in time tag"
        else:
            return "No article-date div found"

    except requests.exceptions.RequestException as e:
        return f"Request error: {e}"
    except Exception as e:
        return f"Error: {e}"

# Test the function with a single URL
test_url = "https://www.airedesantafe.com.ar/policiales/a-un-ano-la-ley-microtrafico-santa-fe-se-derrumbaron-45-bunkeres-droga-y-hubo-mas-100-condenas-toda-la-provincia-n604894"
print(test_date_extraction(test_url))



2024-12-29T14:46:00.000Z


In [123]:
def extract_content(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.content.decode('utf-8', errors='ignore'), 'html.parser')
        
        # Extract paragraphs
        paragraphs = soup.findAll('p')
        content_list = [p.get_text() for p in paragraphs if p.get_text()]

        # Return extracted content
        return content_list

    except requests.exceptions.RequestException as e:
        return [f"Request error: {e}"]
    except Exception as e:
        return [f"Error: {e}"]

# Test the extraction on a single URL
url = "https://www.airedesantafe.com.ar/policiales/a-un-ano-la-ley-microtrafico-santa-fe-se-derrumbaron-45-bunkeres-droga-y-hubo-mas-100-condenas-toda-la-provincia-n604894"  # Replace with your URL
content = extract_content(url)

# Print the extracted content to check the results
print(content)

['Así lo indican desde el Ministerio Público de la Acusación que, mediante un video institucional, informaron los resultados sobre su intervención en casos de narcomenudeo.', 'En mayo de este año, fue derrumbado en barrio Barranquitas el primer búnker de drogas de Santa Fe.', 'Este domingo se cumple un año de la sanción de la ley que delegó en el Ministerio Público de la Acusación las investigaciones sobre casos de  microtráfico de drogas en toda la provincia de Santa Fe.', 'Al respecto, desde la Fiscalía General del MPA, informaron que desde la puesta en vigencia de la nueva normativa se ejecutaron alrededor de 500 allanamientos; 900 audiencias; y un total de 4 mil hombres y 1700 mujeres se encuentran siendo investigadas por maniobras de comercialización de estupefacientes al menudeo.', 'Desde el organismo judicial indicaron además que más de cien personas ya fueron condenadas y se derribaron más de 45 búnkeres en donde se vendían drogas.', 'LEER MÁS ► Derrumbaron por primera vez un b

In [124]:
len(content)

20

In [128]:
content[0]

'Así lo indican desde el Ministerio Público de la Acusación que, mediante un video institucional, informaron los resultados sobre su intervención en casos de narcomenudeo.'